# Enunciado 

Un resorte helicoidal de compresión fabricado, hecho de alambre de piano, tiene un tamaño de alambre de 0.092 in, un diámetro exterior de la espira de 9/16 in, una longitud libre de 4 3/8 in, 21 espiras activas y ambos extremos a escuadra y esmerilados. El resorte está sin granallar, habrá de ensamblarse con una precarga de 5 lb y operará sometido a una carga máxima de 35 lb durante el servicio.

1. Calcule el factor de seguridad que protege contra la falla por fatiga, mediante un lugar geométrico de fatiga por torsión de Gerber y datos de Zimmerli.

2. Repita el inciso *1* usando el lugar geométrico de fatiga por torsión de Sines (la componente del esfuerzo constante no tiene efecto) y los datos de Zimmerli.

3. Repita utilizando el lugar geométrico de falla por torsión de Goodman y datos de Zimmerli

4. Repita utilizando el lugar geométrico de falla de Whal, con datos de Zimmerli.
5. Determine la frecuencia crítica del resorte

### Import necessary libraries

In [78]:
import pandas as pd
from sympy import Eq, solve, Symbol
import numpy as np
from colorama import Fore

### Establecer datos conocidos

In [103]:
d = 0.092 # in
De = 9 / 16  # in
L_f = (4 * 8 + 3) / 8
F_max = 35 #lb
F_min = 5 #lb
Na = 21


### Selección del material

In [80]:
table_10_5 = pd.read_csv('tables/table_10_5.csv')
table_10_5.head()

,Material,Limite tension,limite torsion,Diámetro in,E M-psi,E GPa,G M-psi,G GPa
0,Alambre de piano A228,65-75,45-60,<0.032,29.5,203.4,12.00,82.7
1,Alambre de piano A228,65-75,45-60,0.033-0.063,29.0,200.0,11.85,81.7
2,Alambre de piano A228,65-75,45-60,0.064-0.125,28.5,196.5,11.75,81.0
3,Alambre de piano A228,65-75,45-60,>0.125,28.0,193.0,11.60,80.0
4,Resorte estirado duro A227,60-70,45-55,<0.032,28.8,198.6,11.70,80.7


In [81]:
# Search Cromo-vanadio A231	
count = 2
material  = table_10_5[table_10_5['Material'] == 'Alambre de piano A228'].\
    values[count][1:]
G = material[-2]
porcentaje_sut = int(material[1].split('-')[0])/100
material, porcentaje_sut, G

(array(['65-75', '45-60', '0.064-0.125', 28.5, 196.5, 11.75, 81.0],
       dtype=object),
 0.45,
 11.75)

### Encontrar c

In [82]:
Dm = De - d # in
print(Fore.LIGHTCYAN_EX + 'Dm = ', Dm, 'in', Fore.RESET)

c = Dm/d 
print(Fore.LIGHTGREEN_EX + 'c = ', c, Fore.RESET)


Dm =  0.47050000000000003 in 
c =  5.114130434782609 


### Encontrar el valor de K_b

In [83]:
k_b = (4 * c + 2) / (4 * c - 3)
print(Fore.LIGHTWHITE_EX + 'k_b = ', k_b, Fore.RESET)

k_b =  1.286425902864259 


### Encontrar las fuerzas estándares

In [84]:
F_a = (F_max - F_min) / 2

F_m = (F_max + F_min) /2

print('F_a' + Fore.LIGHTBLUE_EX + ' = ', F_a, 'lb', Fore.RESET)
print('F_m' + Fore.LIGHTBLUE_EX + ' = ', F_m, 'lb', Fore.RESET)

F_a =  15.0 lb 
F_m =  20.0 lb 


### Encontrar los esfuerzos alternates y medio

In [85]:
tau = lambda K_b, F, D_m, d : 8 * K_b * F * D_m / (np.pi * pow(d,3))

tau_a = tau(k_b, F_a, Dm, d)
tau_m = tau(k_b, F_m, Dm, d)

print('tau_a' + Fore.LIGHTBLUE_EX + ' = ', tau_a, 'psi', Fore.RESET)
print('tau_m' + Fore.LIGHTBLUE_EX + ' = ', tau_m, 'psi', Fore.RESET)

tau_a =  29690.1434086965 psi 
tau_m =  39586.857878261995 psi 


### Encontrar la resistencia ultima a la tracción

In [86]:
table_10_4 = pd.read_csv('tables/table_10_4.csv')
table_10_4.head()

,Material,ASTM num,exponente m,diametro in,A kpsi * in^m,Diametro mm,A Mpa * mm^m,Costo relativo,diametro bajo,diametro alto
0,Alambre de piano,A228,0.145,0.004-0.256,201,0.10-6.5,2211,2.6,0.004,0.256
1,Alambre T y R en aceite,A229,0.187,0.020-0.500,147,0.5-12.7,1 855,1.3,0.020,0.500
2,Alambre estirado duro,A227,0.190,0.028-0.500,140,0.7-12.7,1 783,1.0,0.028,0.500
3,Alambre al cromo- vanadio,A232,0.168,0.032-0.437,169,0.8-11.1,2 005,3.1,0.032,0.437
4,Alambre al cromo- silicio,A401,0.108,0.063-0.375,202,1.6-95,1974,40,0.063,0.375


In [87]:
seleccion = table_10_4[
    (table_10_4['ASTM num'] == 'A228' )
    & (d < table_10_4['diametro alto']) 
    & (d > table_10_4['diametro bajo'])
].values[0]

In [88]:
m = seleccion[2]
A = seleccion[4]
print(f'm = {m :,.3f} ', f'A = {A :,.2f} kpi * in^m')

S_ut = A / pow(d, m)
print(f'S_ut = {S_ut :,.5f} kpsi')

m = 0.145  A = 201.00 kpi * in^m
S_ut = 284.08403 kpsi


### Encontrar la resistencia ultima al cortante

In [89]:
S_su = 0.67 * S_ut
print(f'S_su = {S_su :,.5f} kpsi')

S_su = 190.33630 kpsi


### Encontrar la pendiente r

In [90]:
r = tau_a / tau_m
print(f'r = {r :,.3f}')

r = 0.750


### Establecer resistencia constructiva

In [91]:
S_se = 35 / (1 - pow(55 / S_su, 2))
print('S_se' + Fore.LIGHTMAGENTA_EX, S_se, 'kpsi', Fore.RESET)

S_se 38.18872676946134 kpsi 


In [92]:
S_sa = (pow(r * S_su , 2) /(2 * S_se)) * \
    (-1 + pow( 1 + pow(2 * S_se/ (r * S_su) ,2), 1/2)) * 1e3

print('S_sa = ' + Fore.LIGHTCYAN_EX , S_sa, 'psi', Fore.RESET)

S_sa =  35788.47992618446 psi 


In [93]:
n_f = S_sa / tau_a

print('factor de seguridad n_f = ' + Fore.LIGHTYELLOW_EX , n_f, Fore.RESET)

factor de seguridad n_f =  1.205399362123044 


### método de Sines

In [94]:
S_sa = 35e3 #psi
n_f = S_sa / tau_a

print('factor de seguridad n_f = ' + Fore.LIGHTYELLOW_EX , n_f, Fore.RESET)

factor de seguridad n_f =  1.1788424029554603 


### método de goodman

In [96]:
S_se = 35 / (1 - 55/S_su)

S_sa = 1 / (1/S_se + 1/ (r*S_su)) * 1e3
n_f = S_sa / tau_a

print('factor de seguridad n_f = ' + Fore.LIGHTYELLOW_EX , n_f, Fore.RESET)

factor de seguridad n_f =  1.2328178233170104 


### Encontrar el valor del indice del resorte

In [106]:
k = pow(d, 4) * G /(8 * pow(Dm, 3) * Na) * 1e6
print('indice del resorte', Fore.LIGHTYELLOW_EX, k, 'Lb/in2', Fore.RESET)

indice del resorte  48.106195398879855 Lb/in2 


### Encontrar el valor del peso

In [108]:
weight = (pow(d * np.pi, 2) * Dm * Na * 0.2817) / (4)
print('W' + Fore.LIGHTCYAN_EX, weight, 'lb', Fore.RESET)

W 0.0581274375493874 lb 


### Encontrar la frecuencia

In [109]:
f = 1/2 * pow( k * 386.2 /weight, 1/2)
print('f' + Fore.LIGHTCYAN_EX, f, 'Hz', Fore.RESET)

f 282.67412429416095 Hz 


In [110]:
f/F_m

14.133706214708047